In [1]:
# move extracted game's assets to lr and hr folder
import os
from os.path import join
import shutil
root_folder = 'R:/ev'
hr_folder = 'E:/Dataset/hamidashi/hr'
lr_folder = 'E:/Dataset/hamidashi/lr'

for folder_path in [lr_folder, hr_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Use os.walk() to traverse through all directories and subdirectories
for root, dirs, files in os.walk(root_folder):
    for file in files:
        if "png" in file and file[0] == 'z':
            name = file[1:]
            if name not in files:
                continue
            shutil.move(join(root, file), join(hr_folder, name))
            shutil.move(join(root, name), join(lr_folder, name))


In [3]:
import os
from os.path import join
from PIL import Image


def blend_and_save(base, overlay, save):
    base_img = Image.open(base)
    overlay_img = Image.open(overlay)
    x1, y1, w1, h1 = [int(i) for i in base_img.info["comment"].split(",")[1:]]
    x2, y2, w2, h2 = [int(i) for i in overlay_img.info["comment"].split(",")[1:]]
    base_0_img = Image.new(mode="RGB", size=(w1, h1), color=(100, 100, 100))
    base_0_img.paste(base_img, (0, 0), base_img)
    base_0_img.paste(overlay_img, (x2 - x1, y2 - y1), overlay_img)
    base_0_img.save(save)
    base_img.close()
    overlay_img.close()
    base_0_img.close()


fg_folder = "R:\\hamidashi_dump\\totsu\\image\\fg\\"
sabun = dict.fromkeys(os.listdir(fg_folder), "")
output_folder = "R:\\totsu"

# for root, dirs, files in os.walk(fg_folder):
#     if dirs:
#         continue
#     char, size = root.split("\\")[-2:]
#     if sabun[char]:
#         continue
#     list_b_files = [i for i in files if i[0] == "b"]
#     if list_b_files:
#         sabun[char] = max(
#             list_b_files, key=lambda filename: os.path.getsize(join(root, filename))
#         )[1:]
#     else:
#         sabun[char] = max(
#             [i for i in files if i[0] == "a"],
#             key=lambda filename: os.path.getsize(join(root, filename)),
#         )[1:]

# print(sabun)

sabun = {
    "ame": "0017.png",
    "asu": "0051h.png",
    "hiy": "0051.png",
    "kan": "0038h.png",
    "shi": "0007.png",
    "ina": "0046.png",
    "kna": "0051h.png",
    "hir": "0008g.png",
    "mir": "0008h.png",
    "rir": "0008h.png",
    "suk": "0003.png",
    "yuk": "0008h.png",
}

for i in ["no", "z1", "z2"]:
    os.makedirs(join(output_folder, i), exist_ok=True)

for root, dirs, files in os.walk(fg_folder):
    if dirs:
        continue
    char, size = root.split("\\")[-2:]
    if size == "fa":
        continue
    for file in files:
        if char in file:
            blend_and_save(
                join(root, file),
                join(root, file[6] + sabun[char]),
                join(output_folder, size, char + "_" + file[6:]),
            )

In [2]:
import os
from os.path import join
from PIL import Image
import cv2
import numpy as np

hr_folder = "R:/totsu/Output/hr_no/"
lr_folder = "R:/totsu/Output/lr_no/"
size = "no"
# hr_folder = "R:/totsu/Output/hr_z1/"
# lr_folder = "R:/totsu/Output/lr_z1/"
# size = "z1"
hr_cut_folder = "R:/totsu/hr2/"
lr_cut_folder = "R:/totsu/lr2/"
tile_size = 384 // 2
thresdhold = 10

for folder_path in [hr_cut_folder, lr_cut_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

for root, dirs, files in os.walk(lr_folder):
    for file in files:
        base, ext = os.path.splitext(file)
        img_hr = cv2.imread(join(hr_folder, file))
        img_lr = cv2.imread(join(lr_folder, file))
        gray = cv2.cvtColor(img_lr, cv2.COLOR_BGR2GRAY)  # Converting BGR to gray
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        out = np.abs(laplacian)
        count = 0
        while True:
            convolved = cv2.filter2D(
                out,
                -1,
                np.ones((tile_size, tile_size), dtype=np.float64),
                anchor=(0, 0),
                borderType=cv2.BORDER_CONSTANT,
            )
            if (average := np.max(convolved) / (tile_size**2)) < 10:
                print((f"{base} {count+1}"))
                break
            y, x = np.unravel_index(np.argmax(convolved), convolved.shape)
            if x > img_lr.shape[1] - tile_size:
                x = img_lr.shape[1] - tile_size
            if y > img_lr.shape[0] - tile_size:
                y = img_lr.shape[0] - tile_size
            # print(f"{base}_{count}.png {average}")
            out[y : y + tile_size, x : x + tile_size] = 0
            cutted_hr = img_hr[
                y * 2 : y * 2 + tile_size * 2, x * 2 : x * 2 + tile_size * 2
            ]
            cutted_lr = img_lr[y : y + tile_size, x : x + tile_size]
            cv2.imwrite(join(hr_cut_folder, f"{base}_{size}_{count}.png"), cutted_hr)
            cv2.imwrite(join(lr_cut_folder, f"{base}_{size}_{count}.png"), cutted_lr)
            count += 1

ame_a0000 3
ame_a0010 3
ame_a0200 5
ame_a0210 5
ame_a0500 4
ame_a0510 4
ame_a0800 5
ame_a0900 4
ame_a1000 4
ame_a1010 4
ame_a1100 4
ame_a1110 4
ame_b0000 2
ame_b0010 3
ame_b0200 5
ame_b0210 5
ame_b0500 4
ame_b0510 4
ame_b0800 5
ame_b0900 3
ame_b1000 3
ame_b1010 3
ame_b1100 2
ame_b1110 2
asu_a0000 2
asu_a0001 2
asu_a0010 2
asu_a0011 2
asu_a0200 5
asu_a0210 5
asu_a0300 5
asu_a0310 5
asu_a0500 5
asu_a0510 5
asu_a0600 5
asu_a0610 4
asu_a0700 3
asu_a0710 3
asu_a0800 5
asu_a0810 5
asu_a0900 4
asu_a0910 4
asu_a1000 5
asu_a1001 5
asu_a1010 5
asu_a1011 5
asu_a1100 4
asu_a1101 4
asu_a1110 4
asu_a1111 5
asu_b0000 2
asu_b0001 2
asu_b0010 2
asu_b0011 2
asu_b0200 4
asu_b0210 5
asu_b0300 4
asu_b0310 5
asu_b0500 4
asu_b0510 5
asu_b0600 4
asu_b0610 5
asu_b0700 3
asu_b0710 3
asu_b0800 4
asu_b0810 5
asu_b0900 4
asu_b0910 3
asu_b1000 4
asu_b1001 4
asu_b1010 4
asu_b1011 4
asu_b1100 3
asu_b1101 3
asu_b1110 3
asu_b1111 3
hir_a0200 3
hir_a0500 3
hir_a0800 7
hiy_a0000 3
hiy_a0010 3
hiy_a0200 6
hiy_a0210 6
hiy_

In [14]:
import os
from os.path import join
import cv2
import numpy as np
import  shutil
l = []

for root, dirs, files in os.walk("R:\hamidashi\lr"):
    for file in files:
        base, ext = os.path.splitext(file)
        img = cv2.imread(join(root, file))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Converting BGR to gray
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        out = np.abs(laplacian)
        if np.average(out) > 30:
            shutil.copy(join(root, file), "R:/output/data/lr/")
            shutil.copy(join("R:\hamidashi\hr", file), "R:/output/data/hr/")
        # print(f"{file} {}")